In [1]:
#  Copyright (c) Microsoft Corporation.
#  Licensed under the MIT License.

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
from typing import Union, Text
import plotly.express as px
import plotly.graph_objects as go
from pprint import pprint
from pathlib import Path

In [3]:
LABEL0 = "Ref($close, -2) / Ref($close, -1) - 1"

## Dataset used for training

#### Get data

In [4]:
from qlib.tests.data import GetData
GetData().qlib_data(exists_skip=True)

2022-05-11 21:13:27.542 | WARNING  | qlib.tests.data:qlib_data:150 - Data already exists: ~/.qlib/qlib_data/cn_data, the data download will be skipped
	If downloading is required: `exists_skip=False` or `change target_dir`


In [5]:
import qlib
qlib.init()

[28730:MainThread](2022-05-11 21:13:27,748) INFO - qlib.Initialization - [config.py:402] - default_conf: client.
[28730:MainThread](2022-05-11 21:13:28,107) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[28730:MainThread](2022-05-11 21:13:28,109) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/Users/igorlimarochaazevedo/.qlib/qlib_data/cn_data')}


### Training dataset

In the dataset used for training, we have for each stock the as indicators: close, volume, high, low, their respective date and more. 

Let's analyze such data for `SH600000`

In [6]:
from qlib.data import D

df = D.features(['SH600000'], ['$close', '$open', '$high', '$low', '$adjclose', '$factor', '$volume', '$change'], start_time='2008-01-01', end_time='2020-05-31')   
df.reset_index(inplace=True)
df.head()

,instrument,datetime,$close,$open,$high,$low,$adjclose,$factor,$volume,$change
0,SH600000,2008-01-02,10.472238,10.364679,10.765576,10.131963,8.363906,0.86015,67285696.0,0.014205
1,SH600000,2008-01-03,9.948137,10.364679,10.472238,9.695865,7.945321,0.86015,108072488.0,-0.050047
2,SH600000,2008-01-04,10.366634,9.963780,10.483970,9.781910,8.279564,0.86015,71205600.0,0.042068
3,SH600000,2008-01-07,10.949405,10.321657,11.039363,10.302101,8.745008,0.86015,116610240.0,0.056216
4,SH600000,2008-01-08,11.027629,10.990473,11.184076,10.951360,8.807483,0.86015,82458224.0,0.007144


### Data preprocessing

Before passing such data into the Algorithm used for prediction such as XGBoost, LSTM, MLP, etc it is needed to define some data structure and to apply some data preprocessing. 

The data structures can be of type `Alpha360` or `Alpha158`, and the processors used to apply some data preprocessing can be seen under the [link](https://qlib.readthedocs.io/en/latest/component/data.html?highlight=csranknorm)

For this document we will focus on `Alpha360` and the processor `CSRankNorm`.

### Alpha360 definition



When using Alpha360 dataset scheme as the documentation says,

> Alpha360 tries to provide a dataset with original price data
        the original price data includes the prices and volume in the last 60 days.
        To make it easier to learn models from this dataset, all the prices and volume
        are normalized by the latest price and volume data ( dividing by `$close`, `$volume`)
        So the latest normalized `$close` will be 1 (with name CLOSE0), the latest normalized `$volume` will be 1 (with name VOLUME0)
         If further normalization are executed (e.g. centralization),  CLOSE0 and VOLUME0 will be 0.

The dataframe (`df_setup_data`) in the cell <b>below</br> is an example of a dataset used for model training when defining `Alpha360` structure and `CSRankNorm` processor.

In order to obtain such data, it is necessary to define within the method `def setup_data(self, enable_cache: bool = False)` inside `qlib/data/dataset/handler.py` file the following command:


`self._data.to_csv("<<path>>/setup_data.csv")`

Thus, the whole method `setup_data` would be,

```
def setup_data(self, enable_cache: bool = False):
        """
        Set Up the data in case of running initialization for multiple time

        It is responsible for maintaining following variable
        1) self._data

        Parameters
        ----------
        enable_cache : bool
            default value is false:

            - if `enable_cache` == True:

                the processed data will be saved on disk, and handler will load the cached data from the disk directly
                when we call `init` next time
        """
        # Setup data.
        # _data may be with multiple column index level. The outer level indicates the feature set name
        with TimeInspector.logt("Loading data"):
            # make sure the fetch method is based on a index-sorted pd.DataFrame
            self._data = lazy_sort_index(self.data_loader.load(self.instruments, self.start_time, self.end_time))
        # TODO: cache
        self._data.to_csv("<<path>>/setup_data.csv")
```

With that method defined, the command `pip install .` can be executed in order to apply the changes made.

For this tutorial the model that will be used is XGBoost, located under the path `examples/benchmarks/XGBoost/workflow_config_xgboost_Alpha360.yaml`

Finally, lets visualize `df_setup_data_`

In [7]:
df_setup_data_ = pd.read_csv('./files/setup_data.csv', nrows=1000, skiprows=1)
df_setup_data_.rename(columns = {'Unnamed: 0':'datetime', 'Unnamed: 1':'instrument'}, inplace = True)
df_setup_data_ = df_setup_data_.iloc[1:]
print(df_setup_data_.shape)
df_setup_data_.head()

(999, 363)


,datetime,instrument,CLOSE59,CLOSE58,CLOSE57,CLOSE56,CLOSE55,CLOSE54,CLOSE53,CLOSE52,CLOSE51,CLOSE50,CLOSE49,CLOSE48,CLOSE47,CLOSE46,CLOSE45,CLOSE44,CLOSE43,CLOSE42,CLOSE41,CLOSE40,CLOSE39,CLOSE38,CLOSE37,CLOSE36,CLOSE35,CLOSE34,CLOSE33,CLOSE32,CLOSE31,CLOSE30,CLOSE29,CLOSE28,CLOSE27,CLOSE26,CLOSE25,CLOSE24,CLOSE23,CLOSE22,CLOSE21,CLOSE20,CLOSE19,CLOSE18,CLOSE17,CLOSE16,CLOSE15,CLOSE14,CLOSE13,CLOSE12,CLOSE11,CLOSE10,CLOSE9,CLOSE8,CLOSE7,CLOSE6,CLOSE5,CLOSE4,CLOSE3,CLOSE2,CLOSE1,CLOSE0,OPEN59,OPEN58,OPEN57,OPEN56,OPEN55,OPEN54,OPEN53,OPEN52,OPEN51,OPEN50,OPEN49,OPEN48,OPEN47,OPEN46,OPEN45,OPEN44,OPEN43,OPEN42,OPEN41,OPEN40,OPEN39,OPEN38,OPEN37,OPEN36,OPEN35,OPEN34,OPEN33,OPEN32,OPEN31,OPEN30,OPEN29,OPEN28,OPEN27,OPEN26,OPEN25,OPEN24,OPEN23,OPEN22,OPEN21,OPEN20,OPEN19,OPEN18,OPEN17,OPEN16,OPEN15,OPEN14,OPEN13,OPEN12,OPEN11,OPEN10,OPEN9,OPEN8,OPEN7,OPEN6,OPEN5,OPEN4,OPEN3,OPEN2,OPEN1,OPEN0,HIGH59,HIGH58,HIGH57,HIGH56,HIGH55,HIGH54,HIGH53,HIGH52,HIGH51,HIGH50,HIGH49,HIGH48,HIGH47,HIGH46,HIGH45,HIGH44,HIGH43,HIGH42,HIGH41,HIGH40,HIGH39,HIGH38,HIGH37,HIGH36,HIGH35,HIGH34,HIGH33,HIGH32,HIGH31,HIGH30,HIGH29,HIGH28,HIGH27,HIGH26,HIGH25,HIGH24,HIGH23,HIGH22,HIGH21,HIGH20,HIGH19,HIGH18,HIGH17,HIGH16,HIGH15,HIGH14,HIGH13,HIGH12,HIGH11,HIGH10,HIGH9,HIGH8,HIGH7,HIGH6,HIGH5,HIGH4,HIGH3,HIGH2,HIGH1,HIGH0,LOW59,LOW58,LOW57,LOW56,LOW55,LOW54,LOW53,LOW52,LOW51,LOW50,LOW49,LOW48,LOW47,LOW46,LOW45,LOW44,LOW43,LOW42,LOW41,LOW40,LOW39,LOW38,LOW37,LOW36,LOW35,LOW34,LOW33,LOW32,LOW31,LOW30,LOW29,LOW28,LOW27,LOW26,LOW25,LOW24,LOW23,LOW22,LOW21,LOW20,LOW19,LOW18,LOW17,LOW16,LOW15,LOW14,LOW13,LOW12,LOW11,LOW10,LOW9,LOW8,LOW7,LOW6,LOW5,LOW4,LOW3,LOW2,LOW1,LOW0,VWAP59,VWAP58,VWAP57,VWAP56,VWAP55,VWAP54,VWAP53,VWAP52,VWAP51,VWAP50,VWAP49,VWAP48,VWAP47,VWAP46,VWAP45,VWAP44,VWAP43,VWAP42,VWAP41,VWAP40,VWAP39,VWAP38,VWAP37,VWAP36,VWAP35,VWAP34,VWAP33,VWAP32,VWAP31,VWAP30,VWAP29,VWAP28,VWAP27,VWAP26,VWAP25,VWAP24,VWAP23,VWAP22,VWAP21,VWAP20,VWAP19,VWAP18,VWAP17,VWAP16,VWAP15,VWAP14,VWAP13,VWAP12,VWAP11,VWAP10,VWAP9,VWAP8,VWAP7,VWAP6,VWAP5,VWAP4,VWAP3,VWAP2,VWAP1,VWAP0,VOLUME59,VOLUME58,VOLUME57,VOLUME56,VOLUME55,VOLUME54,VOLUME53,VOLUME52,VOLUME51,VOLUME50,VOLUME49,VOLUME48,VOLUME47,VOLUME46,VOLUME45,VOLUME44,VOLUME43,VOLUME42,VOLUME41,VOLUME40,VOLUME39,VOLUME38,VOLUME37,VOLUME36,VOLUME35,VOLUME34,VOLUME33,VOLUME32,VOLUME31,VOLUME30,VOLUME29,VOLUME28,VOLUME27,VOLUME26,VOLUME25,VOLUME24,VOLUME23,VOLUME22,VOLUME21,VOLUME20,VOLUME19,VOLUME18,VOLUME17,VOLUME16,VOLUME15,VOLUME14,VOLUME13,VOLUME12,VOLUME11,VOLUME10,VOLUME9,VOLUME8,VOLUME7,VOLUME6,VOLUME5,VOLUME4,VOLUME3,VOLUME2,VOLUME1,VOLUME0,"Ref($close, -2) / Ref($close, -1) - 1"
1,2008-01-02,SH600000,1.026704,1.010831,1.045191,1.026891,1.007656,1.028385,1.022036,0.964519,0.956302,0.971055,0.982073,1.026517,1.028385,1.084407,1.084034,1.078431,1.097852,1.114472,1.080672,1.025397,1.012138,1.039776,0.988422,1.002428,1.035294,0.987488,1.006723,1.022969,0.987115,0.948646,0.938935,0.921755,0.876190,0.896359,0.892997,0.890570,0.895612,0.965639,0.969001,1.002054,0.992904,1.007656,1.041457,1.038282,1.013259,0.990850,0.932400,0.905135,0.923809,0.868534,0.859384,0.884594,0.922689,0.917461,0.935014,0.938189,0.935387,0.963025,0.985994,1.0,1.028945,1.027077,1.013259,1.045565,1.027077,1.008590,1.025397,1.019608,0.974416,0.950140,0.968814,0.991597,1.021289,1.023343,1.074696,1.084220,1.098973,1.086834,1.100840,1.060691,1.020355,1.017741,1.040150,0.978525,0.979085,1.045191,0.993464,1.005602,1.008403,0.983380,0.943044,0.941363,0.915033,0.868347,0.906816,0.890570,0.896919,0.912792,0.967320,0.959103,1.002614,0.989729,1.009337,1.042017,1.010458,1.014006,0.975537,0.917273,0.896919,0.914846,0.863679,0.868534,0.894491,0.916900,0.926424,0.937442,0.938749,0.934080,0.961718,0.989729,1.045938,1.062185,1.048553,1.071522,1.047619,1.047619,1.066293,1.022035,0.986555,0.977031,0.999066,1.051354,1.053221,1.092437,1.137442,1.096172,1.107190,1.121942,1.116527,1.079365,1.036415,1.042017,1.040150,1.015313,1.041643,1.052661,1.013072,1.045565,1.022969,0

In order to understand how the equation `Ref($close, -2) / Ref($close, -1) - 1` separates each stock on each day, let's visualize the chart below.

The stocks with color yellow are the ones with biggest return and the stocks with purple color are the ones with lowest return.

In [8]:
df_setup_data_filtered = df_setup_data_[(df_setup_data_['datetime'] < '2008-01-06')]
fig = px.scatter(df_setup_data_filtered, x="datetime", y=LABEL0, color=LABEL0, 
                hover_data=["instrument"], 
                labels={
                    "datetime":"Date",
                    LABEL0: "Ref($close, -2) / Ref($close, -1) - 1"
                },
                title="CLOSE0 by day for each instrument")
                
fig.update_xaxes(type='category')
fig.show()

### But how is LABEL0 calculated?

The formula used to calculate such column, using Qlib data retrieval interface, is 

`Ref($close, -2)/Ref($close, -1) - 1`

Where `Ref($close, -2)` will get stock's close value two days in the future. And `Ref($close, -1)` one day in the future. 
The explanation for such equation can be seen under the documentation,

> Qlib uses the label `Ref($close, -2)/Ref($close, -1) - 1` that means the change from T+1 to T+2, rather than `Ref($close, -1)/$close - 1`, of which the reason is that when getting the T day close price of a china stock, the stock can be bought on T+1 day and sold on T+2 day.

Lets calculate such `LABEL0` ( `Ref($close, -2)/Ref($close, -1) - 1` ) for our first ticker (SH600000) for the date 2008-01-02 in order to better undertand how this formula works

In [9]:
# For the instrument SH600000
var_label0 = (df.shift(-2)["$close"]/df.shift(-1)["$close"] - 1)[0]
print("LABEL0 [ Ref($close, -2)/Ref($close, -1) - 1 ] for 2008-01-02:", var_label0)

LABEL0 [ Ref($close, -2)/Ref($close, -1) - 1 ] for 2008-01-02: 0.042067885


It can be calculated in a more intuitive way as well,

In [10]:
# For the instrument SH600000
df_2008_01_03 = df.loc[df["datetime"] == "2008-01-03"]
df_2008_01_04 = df.loc[df["datetime"] == "2008-01-04"]
var_label0 = float(df_2008_01_04["$close"])/float(df_2008_01_03["$close"]) - 1
print("LABEL0 [ Ref($close, -2)/Ref($close, -1) - 1 ] for 2008-01-02:", var_label0)


LABEL0 [ Ref($close, -2)/Ref($close, -1) - 1 ] for 2008-01-02: 0.04206788403219597


Great, now that we understand how `LABEL0` (`Ref($close, -2)/Ref($close, -1) - 1`) is calculated lets take a look into the dataset used for training our model `XGBoost`

In order to obtain the dataset used for training for our model, add the following line into `qlib/contrib/model/xgboost.py` file

`df_train.to_csv("<<path>>/df_train.csv")`

PS: don't forget to run `pip install .` in order to apply the changes made

In [11]:
df_train = pd.read_csv('./files/df_train.csv', nrows=600, skiprows=1)
df_train.rename(columns = {'Unnamed: 0':'datetime', 'Unnamed: 1':'instrument'}, inplace = True)
df_train = df_train.iloc[1:]
print(df_train.shape)
df_train.head()

(599, 363)


,datetime,instrument,CLOSE59,CLOSE58,CLOSE57,CLOSE56,CLOSE55,CLOSE54,CLOSE53,CLOSE52,CLOSE51,CLOSE50,CLOSE49,CLOSE48,CLOSE47,CLOSE46,CLOSE45,CLOSE44,CLOSE43,CLOSE42,CLOSE41,CLOSE40,CLOSE39,CLOSE38,CLOSE37,CLOSE36,CLOSE35,CLOSE34,CLOSE33,CLOSE32,CLOSE31,CLOSE30,CLOSE29,CLOSE28,CLOSE27,CLOSE26,CLOSE25,CLOSE24,CLOSE23,CLOSE22,CLOSE21,CLOSE20,CLOSE19,CLOSE18,CLOSE17,CLOSE16,CLOSE15,CLOSE14,CLOSE13,CLOSE12,CLOSE11,CLOSE10,CLOSE9,CLOSE8,CLOSE7,CLOSE6,CLOSE5,CLOSE4,CLOSE3,CLOSE2,CLOSE1,CLOSE0,OPEN59,OPEN58,OPEN57,OPEN56,OPEN55,OPEN54,OPEN53,OPEN52,OPEN51,OPEN50,OPEN49,OPEN48,OPEN47,OPEN46,OPEN45,OPEN44,OPEN43,OPEN42,OPEN41,OPEN40,OPEN39,OPEN38,OPEN37,OPEN36,OPEN35,OPEN34,OPEN33,OPEN32,OPEN31,OPEN30,OPEN29,OPEN28,OPEN27,OPEN26,OPEN25,OPEN24,OPEN23,OPEN22,OPEN21,OPEN20,OPEN19,OPEN18,OPEN17,OPEN16,OPEN15,OPEN14,OPEN13,OPEN12,OPEN11,OPEN10,OPEN9,OPEN8,OPEN7,OPEN6,OPEN5,OPEN4,OPEN3,OPEN2,OPEN1,OPEN0,HIGH59,HIGH58,HIGH57,HIGH56,HIGH55,HIGH54,HIGH53,HIGH52,HIGH51,HIGH50,HIGH49,HIGH48,HIGH47,HIGH46,HIGH45,HIGH44,HIGH43,HIGH42,HIGH41,HIGH40,HIGH39,HIGH38,HIGH37,HIGH36,HIGH35,HIGH34,HIGH33,HIGH32,HIGH31,HIGH30,HIGH29,HIGH28,HIGH27,HIGH26,HIGH25,HIGH24,HIGH23,HIGH22,HIGH21,HIGH20,HIGH19,HIGH18,HIGH17,HIGH16,HIGH15,HIGH14,HIGH13,HIGH12,HIGH11,HIGH10,HIGH9,HIGH8,HIGH7,HIGH6,HIGH5,HIGH4,HIGH3,HIGH2,HIGH1,HIGH0,LOW59,LOW58,LOW57,LOW56,LOW55,LOW54,LOW53,LOW52,LOW51,LOW50,LOW49,LOW48,LOW47,LOW46,LOW45,LOW44,LOW43,LOW42,LOW41,LOW40,LOW39,LOW38,LOW37,LOW36,LOW35,LOW34,LOW33,LOW32,LOW31,LOW30,LOW29,LOW28,LOW27,LOW26,LOW25,LOW24,LOW23,LOW22,LOW21,LOW20,LOW19,LOW18,LOW17,LOW16,LOW15,LOW14,LOW13,LOW12,LOW11,LOW10,LOW9,LOW8,LOW7,LOW6,LOW5,LOW4,LOW3,LOW2,LOW1,LOW0,VWAP59,VWAP58,VWAP57,VWAP56,VWAP55,VWAP54,VWAP53,VWAP52,VWAP51,VWAP50,VWAP49,VWAP48,VWAP47,VWAP46,VWAP45,VWAP44,VWAP43,VWAP42,VWAP41,VWAP40,VWAP39,VWAP38,VWAP37,VWAP36,VWAP35,VWAP34,VWAP33,VWAP32,VWAP31,VWAP30,VWAP29,VWAP28,VWAP27,VWAP26,VWAP25,VWAP24,VWAP23,VWAP22,VWAP21,VWAP20,VWAP19,VWAP18,VWAP17,VWAP16,VWAP15,VWAP14,VWAP13,VWAP12,VWAP11,VWAP10,VWAP9,VWAP8,VWAP7,VWAP6,VWAP5,VWAP4,VWAP3,VWAP2,VWAP1,VWAP0,VOLUME59,VOLUME58,VOLUME57,VOLUME56,VOLUME55,VOLUME54,VOLUME53,VOLUME52,VOLUME51,VOLUME50,VOLUME49,VOLUME48,VOLUME47,VOLUME46,VOLUME45,VOLUME44,VOLUME43,VOLUME42,VOLUME41,VOLUME40,VOLUME39,VOLUME38,VOLUME37,VOLUME36,VOLUME35,VOLUME34,VOLUME33,VOLUME32,VOLUME31,VOLUME30,VOLUME29,VOLUME28,VOLUME27,VOLUME26,VOLUME25,VOLUME24,VOLUME23,VOLUME22,VOLUME21,VOLUME20,VOLUME19,VOLUME18,VOLUME17,VOLUME16,VOLUME15,VOLUME14,VOLUME13,VOLUME12,VOLUME11,VOLUME10,VOLUME9,VOLUME8,VOLUME7,VOLUME6,VOLUME5,VOLUME4,VOLUME3,VOLUME2,VOLUME1,VOLUME0,"Ref($close, -2) / Ref($close, -1) - 1"
1,2008-01-02,SH600000,1.026704,1.010831,1.045191,1.026891,1.007656,1.028385,1.022036,0.964519,0.956302,0.971055,0.982073,1.026517,1.028385,1.084407,1.084034,1.078431,1.097852,1.114472,1.080672,1.025397,1.012138,1.039776,0.988422,1.002428,1.035294,0.987488,1.006723,1.022969,0.987115,0.948646,0.938935,0.921755,0.876190,0.896359,0.892997,0.890570,0.895612,0.965639,0.969001,1.002054,0.992904,1.007656,1.041457,1.038282,1.013259,0.990850,0.932400,0.905135,0.923809,0.868534,0.859384,0.884594,0.922689,0.917461,0.935014,0.938189,0.935387,0.963025,0.985994,1.0,1.028945,1.027077,1.013259,1.045565,1.027077,1.008590,1.025397,1.019608,0.974416,0.950140,0.968814,0.991597,1.021289,1.023343,1.074696,1.084220,1.098973,1.086834,1.100840,1.060691,1.020355,1.017741,1.040150,0.978525,0.979085,1.045191,0.993464,1.005602,1.008403,0.983380,0.943044,0.941363,0.915033,0.868347,0.906816,0.890570,0.896919,0.912792,0.967320,0.959103,1.002614,0.989729,1.009337,1.042017,1.010458,1.014006,0.975537,0.917273,0.896919,0.914846,0.863679,0.868534,0.894491,0.916900,0.926424,0.937442,0.938749,0.934080,0.961718,0.989729,1.045938,1.062185,1.048553,1.071522,1.047619,1.047619,1.066293,1.022035,0.986555,0.977031,0.999066,1.051354,1.053221,1.092437,1.137442,1.096172,1.107190,1.121942,1.116527,1.079365,1.036415,1.042017,1.040150,1.015313,1.041643,1.052661,1.013072,1.045565,1.022969,0

If you take a look into the `Ref($close, -2) / Ref($close, -1) - 1` column for the instrument `SH600000` at the day `2008-01-02` it shows a value of *1.215436*.

However, we have calculated `Ref($close, -2) / Ref($close, -1) - 1` previously and we obtained *0.04206*. So what happened? Why those values are different?

In order to explain why we obtained different results, lets dive in into another section about _cross sectional rank normalization_ (CSRankNorm)

# cross sectional rank normalization (CSRankNorm)

From the documentation, we have the following definitions:

> CSRankNorm is an operation that grouping the data by each day and rank across all the stocks in each day.

> `CSRankNorm`: processor that applies cross sectional rank normalization.


From statistics we have:

> quantile normalization is a technique for making two distributions identical in statistical properties. 

> To quantile-normalize a test distribution to a reference distribution of the same length, sort the test distribution and sort the reference distribution. The highest entry in the test distribution then takes the value of the highest entry in the reference distribution, the next highest entry in the reference distribution, and so on, until the test distribution is a perturbation of the reference distribution. 

Look at the pandas [reference](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rank.html) to understand how the `rank` function works

In our `workflow_config_xgboost_Alpha360.yaml` file we have defined `class: CSRankNorm` as the code below shows.

```
learn_processors:
        - class: DropnaLabel
        - class: CSRankNorm
          kwargs:
              fields_group: label
```

In the interest of understanding how this CSRankNorm works add the following lines into `qlib/data/dataset/processor.py` file inside the `def __call__(self, df)` method,



```
def __call__(self, df):
    df.to_csv("<<path>>/df_inital_CSRankNorm.csv") #add line
    # try not modify original dataframe
    cols = get_group_columns(df, self.fields_group)
    t = df[cols].groupby("datetime").rank(pct=True)
    t -= 0.5
    t *= 3.46  # NOTE: towards unit std
    df[cols] = t
    df.to_csv("<<path>>/df_CSRankNorm.csv") #add line
    return df
```

Now lets visualize such dataset,

In [12]:
df_inital_CSRankNorm = pd.read_csv('./files/df_inital_CSRankNorm.csv', nrows=600, skiprows=1)
df_inital_CSRankNorm.rename(columns = {'Unnamed: 0':'datetime', 'Unnamed: 1':'instrument'}, inplace = True)
df_inital_CSRankNorm = df_inital_CSRankNorm.iloc[1:]
df_inital_CSRankNorm.head()

,datetime,instrument,CLOSE59,CLOSE58,CLOSE57,CLOSE56,CLOSE55,CLOSE54,CLOSE53,CLOSE52,CLOSE51,CLOSE50,CLOSE49,CLOSE48,CLOSE47,CLOSE46,CLOSE45,CLOSE44,CLOSE43,CLOSE42,CLOSE41,CLOSE40,CLOSE39,CLOSE38,CLOSE37,CLOSE36,CLOSE35,CLOSE34,CLOSE33,CLOSE32,CLOSE31,CLOSE30,CLOSE29,CLOSE28,CLOSE27,CLOSE26,CLOSE25,CLOSE24,CLOSE23,CLOSE22,CLOSE21,CLOSE20,CLOSE19,CLOSE18,CLOSE17,CLOSE16,CLOSE15,CLOSE14,CLOSE13,CLOSE12,CLOSE11,CLOSE10,CLOSE9,CLOSE8,CLOSE7,CLOSE6,CLOSE5,CLOSE4,CLOSE3,CLOSE2,CLOSE1,CLOSE0,OPEN59,OPEN58,OPEN57,OPEN56,OPEN55,OPEN54,OPEN53,OPEN52,OPEN51,OPEN50,OPEN49,OPEN48,OPEN47,OPEN46,OPEN45,OPEN44,OPEN43,OPEN42,OPEN41,OPEN40,OPEN39,OPEN38,OPEN37,OPEN36,OPEN35,OPEN34,OPEN33,OPEN32,OPEN31,OPEN30,OPEN29,OPEN28,OPEN27,OPEN26,OPEN25,OPEN24,OPEN23,OPEN22,OPEN21,OPEN20,OPEN19,OPEN18,OPEN17,OPEN16,OPEN15,OPEN14,OPEN13,OPEN12,OPEN11,OPEN10,OPEN9,OPEN8,OPEN7,OPEN6,OPEN5,OPEN4,OPEN3,OPEN2,OPEN1,OPEN0,HIGH59,HIGH58,HIGH57,HIGH56,HIGH55,HIGH54,HIGH53,HIGH52,HIGH51,HIGH50,HIGH49,HIGH48,HIGH47,HIGH46,HIGH45,HIGH44,HIGH43,HIGH42,HIGH41,HIGH40,HIGH39,HIGH38,HIGH37,HIGH36,HIGH35,HIGH34,HIGH33,HIGH32,HIGH31,HIGH30,HIGH29,HIGH28,HIGH27,HIGH26,HIGH25,HIGH24,HIGH23,HIGH22,HIGH21,HIGH20,HIGH19,HIGH18,HIGH17,HIGH16,HIGH15,HIGH14,HIGH13,HIGH12,HIGH11,HIGH10,HIGH9,HIGH8,HIGH7,HIGH6,HIGH5,HIGH4,HIGH3,HIGH2,HIGH1,HIGH0,LOW59,LOW58,LOW57,LOW56,LOW55,LOW54,LOW53,LOW52,LOW51,LOW50,LOW49,LOW48,LOW47,LOW46,LOW45,LOW44,LOW43,LOW42,LOW41,LOW40,LOW39,LOW38,LOW37,LOW36,LOW35,LOW34,LOW33,LOW32,LOW31,LOW30,LOW29,LOW28,LOW27,LOW26,LOW25,LOW24,LOW23,LOW22,LOW21,LOW20,LOW19,LOW18,LOW17,LOW16,LOW15,LOW14,LOW13,LOW12,LOW11,LOW10,LOW9,LOW8,LOW7,LOW6,LOW5,LOW4,LOW3,LOW2,LOW1,LOW0,VWAP59,VWAP58,VWAP57,VWAP56,VWAP55,VWAP54,VWAP53,VWAP52,VWAP51,VWAP50,VWAP49,VWAP48,VWAP47,VWAP46,VWAP45,VWAP44,VWAP43,VWAP42,VWAP41,VWAP40,VWAP39,VWAP38,VWAP37,VWAP36,VWAP35,VWAP34,VWAP33,VWAP32,VWAP31,VWAP30,VWAP29,VWAP28,VWAP27,VWAP26,VWAP25,VWAP24,VWAP23,VWAP22,VWAP21,VWAP20,VWAP19,VWAP18,VWAP17,VWAP16,VWAP15,VWAP14,VWAP13,VWAP12,VWAP11,VWAP10,VWAP9,VWAP8,VWAP7,VWAP6,VWAP5,VWAP4,VWAP3,VWAP2,VWAP1,VWAP0,VOLUME59,VOLUME58,VOLUME57,VOLUME56,VOLUME55,VOLUME54,VOLUME53,VOLUME52,VOLUME51,VOLUME50,VOLUME49,VOLUME48,VOLUME47,VOLUME46,VOLUME45,VOLUME44,VOLUME43,VOLUME42,VOLUME41,VOLUME40,VOLUME39,VOLUME38,VOLUME37,VOLUME36,VOLUME35,VOLUME34,VOLUME33,VOLUME32,VOLUME31,VOLUME30,VOLUME29,VOLUME28,VOLUME27,VOLUME26,VOLUME25,VOLUME24,VOLUME23,VOLUME22,VOLUME21,VOLUME20,VOLUME19,VOLUME18,VOLUME17,VOLUME16,VOLUME15,VOLUME14,VOLUME13,VOLUME12,VOLUME11,VOLUME10,VOLUME9,VOLUME8,VOLUME7,VOLUME6,VOLUME5,VOLUME4,VOLUME3,VOLUME2,VOLUME1,VOLUME0,"Ref($close, -2) / Ref($close, -1) - 1"
1,2008-01-02,SH600000,1.026704,1.010831,1.045191,1.026891,1.007656,1.028385,1.022036,0.964519,0.956302,0.971055,0.982073,1.026517,1.028385,1.084407,1.084034,1.078431,1.097852,1.114472,1.080672,1.025397,1.012138,1.039776,0.988422,1.002428,1.035294,0.987488,1.006723,1.022969,0.987115,0.948646,0.938935,0.921755,0.876190,0.896359,0.892997,0.890570,0.895612,0.965639,0.969001,1.002054,0.992904,1.007656,1.041457,1.038282,1.013259,0.990850,0.932400,0.905135,0.923809,0.868534,0.859384,0.884594,0.922689,0.917461,0.935014,0.938189,0.935387,0.963025,0.985994,1.0,1.028945,1.027077,1.013259,1.045565,1.027077,1.008590,1.025397,1.019608,0.974416,0.950140,0.968814,0.991597,1.021289,1.023343,1.074696,1.084220,1.098973,1.086834,1.100840,1.060691,1.020355,1.017741,1.040150,0.978525,0.979085,1.045191,0.993464,1.005602,1.008403,0.983380,0.943044,0.941363,0.915033,0.868347,0.906816,0.890570,0.896919,0.912792,0.967320,0.959103,1.002614,0.989729,1.009337,1.042017,1.010458,1.014006,0.975537,0.917273,0.896919,0.914846,0.863679,0.868534,0.894491,0.916900,0.926424,0.937442,0.938749,0.934080,0.961718,0.989729,1.045938,1.062185,1.048553,1.071522,1.047619,1.047619,1.066293,1.022035,0.986555,0.977031,0.999066,1.051354,1.053221,1.092437,1.137442,1.096172,1.107190,1.121942,1.116527,1.079365,1.036415,1.042017,1.040150,1.015313,1.041643,1.052661,1.013072,1.045565,1.022969,0

As you can see the for the column `Ref($close, -2) / Ref($close, -1) - 1` and for the instrument `SH600000` we get the value of _0.042068_. 

Same to value calculated by us previously: _0.04206_.

## How CSRankNorm is calculated

Finally, in order to obtain the value of _1.215436_ for the instrument `SH600000` used for training our model the following formula is used

In [13]:
# First
#   Apply the rank(pct=True) pandas method
print("\n---- (groupby rank pct=True) ----")
print(df_inital_CSRankNorm[['datetime', LABEL0]].groupby("datetime").rank(pct=True).head())

# Second
#   Subtract all values from the column Ref($close, -2) / Ref($close, -1) - 1 by 0..5
print("\n---- (groupby rank pct=True) (- 0.5) ----")
t = df_inital_CSRankNorm[['datetime', LABEL0]].groupby("datetime").rank(pct=True)
t -= 0.5
print(t.head())

# Third
#   Multiply by 3.46 all values from the column Ref($close, -2) / Ref($close, -1) - 1
print("\n---- (groupby rank pct=True) (- 0.5) (*3.46) ----\n")
t *= 3.46
print(t.head())

# Forth
#   Print the first value of our dataframe since it is from SH600000
print("\n---- Finally, the value we've been looking for ----\n")
print(t.iloc[0][0])


---- (groupby rank pct=True) ----
   Ref($close, -2) / Ref($close, -1) - 1
1                               0.851282
2                               0.102564
3                               0.574359
4                               0.076923
5                               0.615385

---- (groupby rank pct=True) (- 0.5) ----
   Ref($close, -2) / Ref($close, -1) - 1
1                               0.351282
2                              -0.397436
3                               0.074359
4                              -0.423077
5                               0.115385

---- (groupby rank pct=True) (- 0.5) (*3.46) ----

   Ref($close, -2) / Ref($close, -1) - 1
1                               1.215436
2                              -1.375128
3                               0.257282
4                              -1.463846
5                               0.399231

---- Finally, the value we've been looking for ----

1.2154358974358974


In [14]:
# Function removed from the file -- qlib/data/dataset/processor.py
def get_group_columns(df: pd.DataFrame, group: Union[Text, None]):
    """
    get a group of columns from multi-index columns DataFrame

    Parameters
    ----------
    df : pd.DataFrame
        with multi of columns.
    group : str
        the name of the feature group, i.e. the first level value of the group index.
    """
    if group is None:
        return df.columns
    else:
        return df.columns[df.columns.get_loc(group)]

From the file `qlib/data/dataset/processor.py` we have the code logic below. 

After executing it, we can finally see that we will obtain for `Ref($close, -2) / Ref($close, -1) - 1` the value of `1.215436` for the instrument `SH600000` at the date `2008-01-02`

In [15]:
cols = get_group_columns(df_inital_CSRankNorm, LABEL0)
t = df_inital_CSRankNorm[['datetime', LABEL0]].groupby("datetime").rank(pct=True)
t -= 0.5
t *= 3.46  # NOTE: towards unit std
df_inital_CSRankNorm[cols] = t
df_inital_CSRankNorm.head()

,datetime,instrument,CLOSE59,CLOSE58,CLOSE57,CLOSE56,CLOSE55,CLOSE54,CLOSE53,CLOSE52,CLOSE51,CLOSE50,CLOSE49,CLOSE48,CLOSE47,CLOSE46,CLOSE45,CLOSE44,CLOSE43,CLOSE42,CLOSE41,CLOSE40,CLOSE39,CLOSE38,CLOSE37,CLOSE36,CLOSE35,CLOSE34,CLOSE33,CLOSE32,CLOSE31,CLOSE30,CLOSE29,CLOSE28,CLOSE27,CLOSE26,CLOSE25,CLOSE24,CLOSE23,CLOSE22,CLOSE21,CLOSE20,CLOSE19,CLOSE18,CLOSE17,CLOSE16,CLOSE15,CLOSE14,CLOSE13,CLOSE12,CLOSE11,CLOSE10,CLOSE9,CLOSE8,CLOSE7,CLOSE6,CLOSE5,CLOSE4,CLOSE3,CLOSE2,CLOSE1,CLOSE0,OPEN59,OPEN58,OPEN57,OPEN56,OPEN55,OPEN54,OPEN53,OPEN52,OPEN51,OPEN50,OPEN49,OPEN48,OPEN47,OPEN46,OPEN45,OPEN44,OPEN43,OPEN42,OPEN41,OPEN40,OPEN39,OPEN38,OPEN37,OPEN36,OPEN35,OPEN34,OPEN33,OPEN32,OPEN31,OPEN30,OPEN29,OPEN28,OPEN27,OPEN26,OPEN25,OPEN24,OPEN23,OPEN22,OPEN21,OPEN20,OPEN19,OPEN18,OPEN17,OPEN16,OPEN15,OPEN14,OPEN13,OPEN12,OPEN11,OPEN10,OPEN9,OPEN8,OPEN7,OPEN6,OPEN5,OPEN4,OPEN3,OPEN2,OPEN1,OPEN0,HIGH59,HIGH58,HIGH57,HIGH56,HIGH55,HIGH54,HIGH53,HIGH52,HIGH51,HIGH50,HIGH49,HIGH48,HIGH47,HIGH46,HIGH45,HIGH44,HIGH43,HIGH42,HIGH41,HIGH40,HIGH39,HIGH38,HIGH37,HIGH36,HIGH35,HIGH34,HIGH33,HIGH32,HIGH31,HIGH30,HIGH29,HIGH28,HIGH27,HIGH26,HIGH25,HIGH24,HIGH23,HIGH22,HIGH21,HIGH20,HIGH19,HIGH18,HIGH17,HIGH16,HIGH15,HIGH14,HIGH13,HIGH12,HIGH11,HIGH10,HIGH9,HIGH8,HIGH7,HIGH6,HIGH5,HIGH4,HIGH3,HIGH2,HIGH1,HIGH0,LOW59,LOW58,LOW57,LOW56,LOW55,LOW54,LOW53,LOW52,LOW51,LOW50,LOW49,LOW48,LOW47,LOW46,LOW45,LOW44,LOW43,LOW42,LOW41,LOW40,LOW39,LOW38,LOW37,LOW36,LOW35,LOW34,LOW33,LOW32,LOW31,LOW30,LOW29,LOW28,LOW27,LOW26,LOW25,LOW24,LOW23,LOW22,LOW21,LOW20,LOW19,LOW18,LOW17,LOW16,LOW15,LOW14,LOW13,LOW12,LOW11,LOW10,LOW9,LOW8,LOW7,LOW6,LOW5,LOW4,LOW3,LOW2,LOW1,LOW0,VWAP59,VWAP58,VWAP57,VWAP56,VWAP55,VWAP54,VWAP53,VWAP52,VWAP51,VWAP50,VWAP49,VWAP48,VWAP47,VWAP46,VWAP45,VWAP44,VWAP43,VWAP42,VWAP41,VWAP40,VWAP39,VWAP38,VWAP37,VWAP36,VWAP35,VWAP34,VWAP33,VWAP32,VWAP31,VWAP30,VWAP29,VWAP28,VWAP27,VWAP26,VWAP25,VWAP24,VWAP23,VWAP22,VWAP21,VWAP20,VWAP19,VWAP18,VWAP17,VWAP16,VWAP15,VWAP14,VWAP13,VWAP12,VWAP11,VWAP10,VWAP9,VWAP8,VWAP7,VWAP6,VWAP5,VWAP4,VWAP3,VWAP2,VWAP1,VWAP0,VOLUME59,VOLUME58,VOLUME57,VOLUME56,VOLUME55,VOLUME54,VOLUME53,VOLUME52,VOLUME51,VOLUME50,VOLUME49,VOLUME48,VOLUME47,VOLUME46,VOLUME45,VOLUME44,VOLUME43,VOLUME42,VOLUME41,VOLUME40,VOLUME39,VOLUME38,VOLUME37,VOLUME36,VOLUME35,VOLUME34,VOLUME33,VOLUME32,VOLUME31,VOLUME30,VOLUME29,VOLUME28,VOLUME27,VOLUME26,VOLUME25,VOLUME24,VOLUME23,VOLUME22,VOLUME21,VOLUME20,VOLUME19,VOLUME18,VOLUME17,VOLUME16,VOLUME15,VOLUME14,VOLUME13,VOLUME12,VOLUME11,VOLUME10,VOLUME9,VOLUME8,VOLUME7,VOLUME6,VOLUME5,VOLUME4,VOLUME3,VOLUME2,VOLUME1,VOLUME0,"Ref($close, -2) / Ref($close, -1) - 1"
1,2008-01-02,SH600000,1.026704,1.010831,1.045191,1.026891,1.007656,1.028385,1.022036,0.964519,0.956302,0.971055,0.982073,1.026517,1.028385,1.084407,1.084034,1.078431,1.097852,1.114472,1.080672,1.025397,1.012138,1.039776,0.988422,1.002428,1.035294,0.987488,1.006723,1.022969,0.987115,0.948646,0.938935,0.921755,0.876190,0.896359,0.892997,0.890570,0.895612,0.965639,0.969001,1.002054,0.992904,1.007656,1.041457,1.038282,1.013259,0.990850,0.932400,0.905135,0.923809,0.868534,0.859384,0.884594,0.922689,0.917461,0.935014,0.938189,0.935387,0.963025,0.985994,1.0,1.028945,1.027077,1.013259,1.045565,1.027077,1.008590,1.025397,1.019608,0.974416,0.950140,0.968814,0.991597,1.021289,1.023343,1.074696,1.084220,1.098973,1.086834,1.100840,1.060691,1.020355,1.017741,1.040150,0.978525,0.979085,1.045191,0.993464,1.005602,1.008403,0.983380,0.943044,0.941363,0.915033,0.868347,0.906816,0.890570,0.896919,0.912792,0.967320,0.959103,1.002614,0.989729,1.009337,1.042017,1.010458,1.014006,0.975537,0.917273,0.896919,0.914846,0.863679,0.868534,0.894491,0.916900,0.926424,0.937442,0.938749,0.934080,0.961718,0.989729,1.045938,1.062185,1.048553,1.071522,1.047619,1.047619,1.066293,1.022035,0.986555,0.977031,0.999066,1.051354,1.053221,1.092437,1.137442,1.096172,1.107190,1.121942,1.116527,1.079365,1.036415,1.042017,1.040150,1.015313,1.041643,1.052661,1.013072,1.045565,1.022969,0

In [16]:
df_inital_CSRankNorm

,datetime,instrument,CLOSE59,CLOSE58,CLOSE57,CLOSE56,CLOSE55,CLOSE54,CLOSE53,CLOSE52,CLOSE51,CLOSE50,CLOSE49,CLOSE48,CLOSE47,CLOSE46,CLOSE45,CLOSE44,CLOSE43,CLOSE42,CLOSE41,CLOSE40,CLOSE39,CLOSE38,CLOSE37,CLOSE36,CLOSE35,CLOSE34,CLOSE33,CLOSE32,CLOSE31,CLOSE30,CLOSE29,CLOSE28,CLOSE27,CLOSE26,CLOSE25,CLOSE24,CLOSE23,CLOSE22,CLOSE21,CLOSE20,CLOSE19,CLOSE18,CLOSE17,CLOSE16,CLOSE15,CLOSE14,CLOSE13,CLOSE12,CLOSE11,CLOSE10,CLOSE9,CLOSE8,CLOSE7,CLOSE6,CLOSE5,CLOSE4,CLOSE3,CLOSE2,CLOSE1,CLOSE0,OPEN59,OPEN58,OPEN57,OPEN56,OPEN55,OPEN54,OPEN53,OPEN52,OPEN51,OPEN50,OPEN49,OPEN48,OPEN47,OPEN46,OPEN45,OPEN44,OPEN43,OPEN42,OPEN41,OPEN40,OPEN39,OPEN38,OPEN37,OPEN36,OPEN35,OPEN34,OPEN33,OPEN32,OPEN31,OPEN30,OPEN29,OPEN28,OPEN27,OPEN26,OPEN25,OPEN24,OPEN23,OPEN22,OPEN21,OPEN20,OPEN19,OPEN18,OPEN17,OPEN16,OPEN15,OPEN14,OPEN13,OPEN12,OPEN11,OPEN10,OPEN9,OPEN8,OPEN7,OPEN6,OPEN5,OPEN4,OPEN3,OPEN2,OPEN1,OPEN0,HIGH59,HIGH58,HIGH57,HIGH56,HIGH55,HIGH54,HIGH53,HIGH52,HIGH51,HIGH50,HIGH49,HIGH48,HIGH47,HIGH46,HIGH45,HIGH44,HIGH43,HIGH42,HIGH41,HIGH40,HIGH39,HIGH38,HIGH37,HIGH36,HIGH35,HIGH34,HIGH33,HIGH32,HIGH31,HIGH30,HIGH29,HIGH28,HIGH27,HIGH26,HIGH25,HIGH24,HIGH23,HIGH22,HIGH21,HIGH20,HIGH19,HIGH18,HIGH17,HIGH16,HIGH15,HIGH14,HIGH13,HIGH12,HIGH11,HIGH10,HIGH9,HIGH8,HIGH7,HIGH6,HIGH5,HIGH4,HIGH3,HIGH2,HIGH1,HIGH0,LOW59,LOW58,LOW57,LOW56,LOW55,LOW54,LOW53,LOW52,LOW51,LOW50,LOW49,LOW48,LOW47,LOW46,LOW45,LOW44,LOW43,LOW42,LOW41,LOW40,LOW39,LOW38,LOW37,LOW36,LOW35,LOW34,LOW33,LOW32,LOW31,LOW30,LOW29,LOW28,LOW27,LOW26,LOW25,LOW24,LOW23,LOW22,LOW21,LOW20,LOW19,LOW18,LOW17,LOW16,LOW15,LOW14,LOW13,LOW12,LOW11,LOW10,LOW9,LOW8,LOW7,LOW6,LOW5,LOW4,LOW3,LOW2,LOW1,LOW0,VWAP59,VWAP58,VWAP57,VWAP56,VWAP55,VWAP54,VWAP53,VWAP52,VWAP51,VWAP50,VWAP49,VWAP48,VWAP47,VWAP46,VWAP45,VWAP44,VWAP43,VWAP42,VWAP41,VWAP40,VWAP39,VWAP38,VWAP37,VWAP36,VWAP35,VWAP34,VWAP33,VWAP32,VWAP31,VWAP30,VWAP29,VWAP28,VWAP27,VWAP26,VWAP25,VWAP24,VWAP23,VWAP22,VWAP21,VWAP20,VWAP19,VWAP18,VWAP17,VWAP16,VWAP15,VWAP14,VWAP13,VWAP12,VWAP11,VWAP10,VWAP9,VWAP8,VWAP7,VWAP6,VWAP5,VWAP4,VWAP3,VWAP2,VWAP1,VWAP0,VOLUME59,VOLUME58,VOLUME57,VOLUME56,VOLUME55,VOLUME54,VOLUME53,VOLUME52,VOLUME51,VOLUME50,VOLUME49,VOLUME48,VOLUME47,VOLUME46,VOLUME45,VOLUME44,VOLUME43,VOLUME42,VOLUME41,VOLUME40,VOLUME39,VOLUME38,VOLUME37,VOLUME36,VOLUME35,VOLUME34,VOLUME33,VOLUME32,VOLUME31,VOLUME30,VOLUME29,VOLUME28,VOLUME27,VOLUME26,VOLUME25,VOLUME24,VOLUME23,VOLUME22,VOLUME21,VOLUME20,VOLUME19,VOLUME18,VOLUME17,VOLUME16,VOLUME15,VOLUME14,VOLUME13,VOLUME12,VOLUME11,VOLUME10,VOLUME9,VOLUME8,VOLUME7,VOLUME6,VOLUME5,VOLUME4,VOLUME3,VOLUME2,VOLUME1,VOLUME0,"Ref($close, -2) / Ref($close, -1) - 1"
1,2008-01-02,SH600000,1.026704,1.010831,1.045191,1.026891,1.007656,1.028385,1.022036,0.964519,0.956302,0.971055,0.982073,1.026517,1.028385,1.084407,1.084034,1.078431,1.097852,1.114472,1.080672,1.025397,1.012138,1.039776,0.988422,1.002428,1.035294,0.987488,1.006723,1.022969,0.987115,0.948646,0.938935,0.921755,0.876190,0.896359,0.892997,0.890570,0.895612,0.965639,0.969001,1.002054,0.992904,1.007656,1.041457,1.038282,1.013259,0.990850,0.932400,0.905135,0.923809,0.868534,0.859384,0.884594,0.922689,0.917461,0.935014,0.938189,0.935387,0.963025,0.985994,1.0,1.028945,1.027077,1.013259,1.045565,1.027077,1.008590,1.025397,1.019608,0.974416,0.950140,0.968814,0.991597,1.021289,1.023343,1.074696,1.084220,1.098973,1.086834,1.100840,1.060691,1.020355,1.017741,1.040150,0.978525,0.979085,1.045191,0.993464,1.005602,1.008403,0.983380,0.943044,0.941363,0.915033,0.868347,0.906816,0.890570,0.896919,0.912792,0.967320,0.959103,1.002614,0.989729,1.009337,1.042017,1.010458,1.014006,0.975537,0.917273,0.896919,0.914846,0.863679,0.868534,0.894491,0.916900,0.926424,0.937442,0.938749,0.934080,0.961718,0.989729,1.045938,1.062185,1.048553,1.071522,1.047619,1.047619,1.066293,1.022035,0.986555,0.977031,0.999066,1.051354,1.053221,1.092437,1.137442,1.096172,1.107190,1.121942,1.116527,1.079365,1.036415,1.042017,1.040150,1.015313,1.041643,1.052661,1.013072,1.045565,1.022969,0

In order to better understand the effect of CSRankNorm our our data, visualize the chart below.

The stocks with yellow color are the ones with biggest return and the stocks with purple color are the ones with lowest return.

In [17]:
df_inital_CSRankNorm_filtered = df_inital_CSRankNorm[(df_inital_CSRankNorm['datetime'] < '2008-01-06')]
fig = px.scatter(df_inital_CSRankNorm_filtered, x="datetime", y=LABEL0, color=LABEL0, 
                hover_data=["instrument"], 
                labels={
                    "datetime":"Date",
                    LABEL0: "Ref(&#36;close, -2) / Ref(&#36;close, -1) - 1"
                },
                title="CLOSE0 by day for each instrument")
                
fig.update_xaxes(type='category')
fig.show()

As we can see the values obtained from CSRankNorm have much less spacing between the points for each ticker when compared with our first chart obtained.



### Explanation for (t -= 0.5) and (t *= 3.46)

#### Analysis (1)

Lets first see the `describe` method for our data by applying the `rank` method but without subtracting the data by 0.5 and by multiplying it by 3.46.

In [18]:
# Without subtracting 0.5 and multiplying by 3.46
# Analysis (1)

cols = get_group_columns(df_inital_CSRankNorm, LABEL0)
t = df_inital_CSRankNorm[['datetime', LABEL0]].groupby("datetime").rank(pct=True)
df_inital_CSRankNorm[cols] = t
df_ = df_inital_CSRankNorm.copy()
df_[LABEL0].describe()

count    599.000000
mean       0.503339
std        0.288929
min        0.005128
25%        0.254547
50%        0.502591
75%        0.752571
max        1.000000
Name: Ref($close, -2) / Ref($close, -1) - 1, dtype: float64

#### Analysis (2)

Now, lets first see the `describe` method for our data by applying the `rank` method and by subtracting our data by 0.5 and by multiplying it by 3.46

In [19]:
# Analysis (2)
# Subtracting 0.5 and multiplying by 3.46

cols = get_group_columns(df_inital_CSRankNorm, LABEL0)
t = df_inital_CSRankNorm[['datetime', LABEL0]].groupby("datetime").rank(pct=True)
t -= 0.5
t *= 3.46  # NOTE: towards unit std
df_inital_CSRankNorm[cols] = t
df_ = df_inital_CSRankNorm.copy()
df_[LABEL0].describe()

count    599.000000
mean       0.011553
std        0.999695
min       -1.712256
25%       -0.849267
50%        0.008964
75%        0.873895
max        1.730000
Name: Ref($close, -2) / Ref($close, -1) - 1, dtype: float64

In a straightforward explanation, we can see that the goal with the operations (t -= 0.5) and (t *= 3.46) is to make our data _closest as possible to a normal distribution_. 

In order words, make the mean closest as possible to 0 and the standard deviation closest as possible to 1. Please, refer to Analysis(1) and Anaylsis(2) to perceive the difference between both.

In [20]:
df_inital_CSRankNorm.iloc[:1000]

,datetime,instrument,CLOSE59,CLOSE58,CLOSE57,CLOSE56,CLOSE55,CLOSE54,CLOSE53,CLOSE52,CLOSE51,CLOSE50,CLOSE49,CLOSE48,CLOSE47,CLOSE46,CLOSE45,CLOSE44,CLOSE43,CLOSE42,CLOSE41,CLOSE40,CLOSE39,CLOSE38,CLOSE37,CLOSE36,CLOSE35,CLOSE34,CLOSE33,CLOSE32,CLOSE31,CLOSE30,CLOSE29,CLOSE28,CLOSE27,CLOSE26,CLOSE25,CLOSE24,CLOSE23,CLOSE22,CLOSE21,CLOSE20,CLOSE19,CLOSE18,CLOSE17,CLOSE16,CLOSE15,CLOSE14,CLOSE13,CLOSE12,CLOSE11,CLOSE10,CLOSE9,CLOSE8,CLOSE7,CLOSE6,CLOSE5,CLOSE4,CLOSE3,CLOSE2,CLOSE1,CLOSE0,OPEN59,OPEN58,OPEN57,OPEN56,OPEN55,OPEN54,OPEN53,OPEN52,OPEN51,OPEN50,OPEN49,OPEN48,OPEN47,OPEN46,OPEN45,OPEN44,OPEN43,OPEN42,OPEN41,OPEN40,OPEN39,OPEN38,OPEN37,OPEN36,OPEN35,OPEN34,OPEN33,OPEN32,OPEN31,OPEN30,OPEN29,OPEN28,OPEN27,OPEN26,OPEN25,OPEN24,OPEN23,OPEN22,OPEN21,OPEN20,OPEN19,OPEN18,OPEN17,OPEN16,OPEN15,OPEN14,OPEN13,OPEN12,OPEN11,OPEN10,OPEN9,OPEN8,OPEN7,OPEN6,OPEN5,OPEN4,OPEN3,OPEN2,OPEN1,OPEN0,HIGH59,HIGH58,HIGH57,HIGH56,HIGH55,HIGH54,HIGH53,HIGH52,HIGH51,HIGH50,HIGH49,HIGH48,HIGH47,HIGH46,HIGH45,HIGH44,HIGH43,HIGH42,HIGH41,HIGH40,HIGH39,HIGH38,HIGH37,HIGH36,HIGH35,HIGH34,HIGH33,HIGH32,HIGH31,HIGH30,HIGH29,HIGH28,HIGH27,HIGH26,HIGH25,HIGH24,HIGH23,HIGH22,HIGH21,HIGH20,HIGH19,HIGH18,HIGH17,HIGH16,HIGH15,HIGH14,HIGH13,HIGH12,HIGH11,HIGH10,HIGH9,HIGH8,HIGH7,HIGH6,HIGH5,HIGH4,HIGH3,HIGH2,HIGH1,HIGH0,LOW59,LOW58,LOW57,LOW56,LOW55,LOW54,LOW53,LOW52,LOW51,LOW50,LOW49,LOW48,LOW47,LOW46,LOW45,LOW44,LOW43,LOW42,LOW41,LOW40,LOW39,LOW38,LOW37,LOW36,LOW35,LOW34,LOW33,LOW32,LOW31,LOW30,LOW29,LOW28,LOW27,LOW26,LOW25,LOW24,LOW23,LOW22,LOW21,LOW20,LOW19,LOW18,LOW17,LOW16,LOW15,LOW14,LOW13,LOW12,LOW11,LOW10,LOW9,LOW8,LOW7,LOW6,LOW5,LOW4,LOW3,LOW2,LOW1,LOW0,VWAP59,VWAP58,VWAP57,VWAP56,VWAP55,VWAP54,VWAP53,VWAP52,VWAP51,VWAP50,VWAP49,VWAP48,VWAP47,VWAP46,VWAP45,VWAP44,VWAP43,VWAP42,VWAP41,VWAP40,VWAP39,VWAP38,VWAP37,VWAP36,VWAP35,VWAP34,VWAP33,VWAP32,VWAP31,VWAP30,VWAP29,VWAP28,VWAP27,VWAP26,VWAP25,VWAP24,VWAP23,VWAP22,VWAP21,VWAP20,VWAP19,VWAP18,VWAP17,VWAP16,VWAP15,VWAP14,VWAP13,VWAP12,VWAP11,VWAP10,VWAP9,VWAP8,VWAP7,VWAP6,VWAP5,VWAP4,VWAP3,VWAP2,VWAP1,VWAP0,VOLUME59,VOLUME58,VOLUME57,VOLUME56,VOLUME55,VOLUME54,VOLUME53,VOLUME52,VOLUME51,VOLUME50,VOLUME49,VOLUME48,VOLUME47,VOLUME46,VOLUME45,VOLUME44,VOLUME43,VOLUME42,VOLUME41,VOLUME40,VOLUME39,VOLUME38,VOLUME37,VOLUME36,VOLUME35,VOLUME34,VOLUME33,VOLUME32,VOLUME31,VOLUME30,VOLUME29,VOLUME28,VOLUME27,VOLUME26,VOLUME25,VOLUME24,VOLUME23,VOLUME22,VOLUME21,VOLUME20,VOLUME19,VOLUME18,VOLUME17,VOLUME16,VOLUME15,VOLUME14,VOLUME13,VOLUME12,VOLUME11,VOLUME10,VOLUME9,VOLUME8,VOLUME7,VOLUME6,VOLUME5,VOLUME4,VOLUME3,VOLUME2,VOLUME1,VOLUME0,"Ref($close, -2) / Ref($close, -1) - 1"
1,2008-01-02,SH600000,1.026704,1.010831,1.045191,1.026891,1.007656,1.028385,1.022036,0.964519,0.956302,0.971055,0.982073,1.026517,1.028385,1.084407,1.084034,1.078431,1.097852,1.114472,1.080672,1.025397,1.012138,1.039776,0.988422,1.002428,1.035294,0.987488,1.006723,1.022969,0.987115,0.948646,0.938935,0.921755,0.876190,0.896359,0.892997,0.890570,0.895612,0.965639,0.969001,1.002054,0.992904,1.007656,1.041457,1.038282,1.013259,0.990850,0.932400,0.905135,0.923809,0.868534,0.859384,0.884594,0.922689,0.917461,0.935014,0.938189,0.935387,0.963025,0.985994,1.0,1.028945,1.027077,1.013259,1.045565,1.027077,1.008590,1.025397,1.019608,0.974416,0.950140,0.968814,0.991597,1.021289,1.023343,1.074696,1.084220,1.098973,1.086834,1.100840,1.060691,1.020355,1.017741,1.040150,0.978525,0.979085,1.045191,0.993464,1.005602,1.008403,0.983380,0.943044,0.941363,0.915033,0.868347,0.906816,0.890570,0.896919,0.912792,0.967320,0.959103,1.002614,0.989729,1.009337,1.042017,1.010458,1.014006,0.975537,0.917273,0.896919,0.914846,0.863679,0.868534,0.894491,0.916900,0.926424,0.937442,0.938749,0.934080,0.961718,0.989729,1.045938,1.062185,1.048553,1.071522,1.047619,1.047619,1.066293,1.022035,0.986555,0.977031,0.999066,1.051354,1.053221,1.092437,1.137442,1.096172,1.107190,1.121942,1.116527,1.079365,1.036415,1.042017,1.040150,1.015313,1.041643,1.052661,1.013072,1.045565,1.022969,0